🛠️ Step 1: Data Extraction & Environment Setup In this section, we initialize the project directory structure (raw/, processed/, output/) and load the raw Customer Churn dataset into our environment.

In [2]:
import os
for folder in ['raw', 'processed', 'output']:
    os.makedirs(folder, exist_ok=True)

In [3]:
import pandas as pd
df = pd.read_csv('raw dataset churn.csv')

🧹 Step 2: Data Transformation & Cleaning We perform data hygiene by checking for duplicates and missing values. We also standardize column names to snake_case and create derived segments for Age and Credit Scores.

In [12]:
df = df.drop_duplicates(subset=['customerid'])
df.isnull().sum()
df.columns = [col.lower() for col in df.columns]

📊 Step 3: Data Normalization & Splitting To optimize data management, we split the flat dataset into three relational tables: dim_customers, dim_bank_details, and fact_churn.

In [13]:
df['age_group'] = df['age'].apply(lambda x: 'Young' if x < 30 else ('Middle-aged' if x < 50 else 'Senior'))

In [14]:
df['credit_segment'] = df['creditscore'].apply(lambda x: 'Poor' if x < 580 else ('Fair' if x < 670 else 'Good'))

In [15]:
dim_customers = df[['customerid', 'surname', 'geography', 'gender', 'age', 'age_group']]

In [16]:
dim_bank = df[['customerid', 'creditscore', 'balance', 'estimatedsalary']]

In [17]:
fact_churn = df[['customerid', 'exited']]

In [18]:
# Save CSVs
dim_customers.to_csv('processed/dim_customers.csv', index=False)
dim_bank.to_csv('processed/dim_bank_details.csv', index=False)
fact_churn.to_csv('processed/fact_churn.csv', index=False)

💾 Step 4: Loading to SQLite Database The final step involves persisting our transformed data into an SQLite database (database.sqlite) and exporting the processed tables as CSV files.

In [22]:
import sqlite3

# Save to SQLite
conn = sqlite3.connect('output/database.sqlite')
dim_customers.to_sql('customers', conn, if_exists='replace', index=False)
dim_bank.to_sql('bank_details', conn, if_exists='replace', index=False)
fact_churn.to_sql('churn_status', conn, if_exists='replace', index=False)
conn.close()

In [23]:
print("ETL Pipeline completed successfully!")

ETL Pipeline completed successfully!


✔ final check


In [24]:
print(f"Original Rows: {len(df)}")
print(f"Customer Table Rows: {len(dim_customers)}")
# These numbers should match!

Original Rows: 10000
Customer Table Rows: 10000


In [25]:
conn = sqlite3.connect('output/database.sqlite')
test_query = pd.read_sql("SELECT * FROM customers LIMIT 5", conn)
print(test_query)
conn.close()

   customerid   surname geography  gender  age    age_group
0    15634602  Hargrave    France  Female   42  Middle-aged
1    15647311      Hill     Spain  Female   41  Middle-aged
2    15619304      Onio    France  Female   42  Middle-aged
3    15701354      Boni    France  Female   39  Middle-aged
4    15737888  Mitchell     Spain  Female   43  Middle-aged
